In [ ]:
import os
import hypso as h

from hypso.experimental.chlorophyll import start_chl_estimation
from hypso.classification.water import ndwi_watermask
from hypso.georeference import generate_full_geotiff as gfg

import utils
from utils import check4files

check4files()

dataPath = os.path.join(os.getcwd(), "florida_2024-04-13_1543Z-l1a.nc")
modelPath = os.path.join(os.getcwd(), "model_6sv1_aqua_tuned.joblib")
pointsPath = os.path.join(os.getcwd(), "florida_2024-04-13_1543Z-rgba_8bit.tif.points")

h_file = h.Hypso(dataPath, points_path=pointsPath)

atmos_params = {
    'aot550': 0.0580000256
}

# AOT550 parameter gotten from: https://giovanni.gsfc.nasa.gov/giovanni/
# In this case, if you force reload, you would delete L1C so we set it to False
h_file.create_geotiff(product="L2-6SV1", force_reload=False, atmos_dict=atmos_params)

h_file.find_existing_l2_cube()

gfg(h_file)
ndwi_watermask(h_file)

start_chl_estimation(sat_obj=h_file, model_path=modelPath)

from hypso.plot import plot_array_overlay
# Plot Array on Map
plot_array_overlay(h_file,h_file.chl, plotTitle="6SV1 Estimation",cbar_title="Chlorophyll Values Sample 0 to 100", min_value=0.01, max_value=100)

In [ ]:
from utils import embed_geo_data

# Path to geo-referenced TIF file
chlFName = "florida_2024-04-13_1543Z-rgba_8bit_chl.tif"
embed_geo_data(chlFName, h_file, h_file.chl)